<a href="https://colab.research.google.com/github/anshupandey/AIOps-EL/blob/main/part3_standard_code_with_mlflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Quality of Service (QoS) Metrics for 5G Network Optimization

In [15]:
!pip install mlflow --quiet

In [16]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import mlflow
import mlflow.sklearn

In [17]:
!wget -q https://www.dropbox.com/scl/fi/0rncal8biwhhw5opxyxz6/Quality-of-Service-5G.xlsx?rlkey=gdoq5hw1fgc60khl3equbreyn&st=uzqgtrca&dl=0
!mv Quality-of-Service-5G.xlsx?rlkey=gdoq5hw1fgc60khl3equbreyn Quality-of-Service-5G.xlsx

In [20]:
# Load the dataset
file_path = 'Quality-of-Service-5G.xlsx'
data = pd.read_excel(file_path, sheet_name='in')
data.shape

(400, 8)

In [21]:
data.head()

,Timestamp,User_ID,Application_Type,Signal_Strength,Latency,Required_Bandwidth,Allocated_Bandwidth,Resource_Allocation
0,2023-09-03 10:00:00,User_1,Video_Call,-75 dBm,30 ms,10 Mbps,15 Mbps,0.70
1,2023-09-03 10:00:00,User_2,Voice_Call,-80 dBm,20 ms,100 Kbps,120 Kbps,0.80
2,2023-09-03 10:00:00,User_3,Streaming,-85 dBm,40 ms,5 Mbps,6 Mbps,0.75
3,2023-09-03 10:00:00,User_4,Emergency_Service,-70 dBm,10 ms,1 Mbps,1.5 Mbps,0.90
4,2023-09-03 10:00:00,User_5,Online_Gaming,-78 dBm,25 ms,2 Mbps,3 Mbps,0.85


# Data preprocessing

In [22]:
# Convert columns to numeric where applicable
data['Signal_Strength'] = data['Signal_Strength'].str.replace(' dBm', '').astype(float)
data['Latency'] = data['Latency'].str.replace(' ms', '').astype(float)
data['Required_Bandwidth'] = data['Required_Bandwidth'].str.replace(' Mbps', '').str.replace(' Kbps', '').astype(float)
data['Allocated_Bandwidth'] = data['Allocated_Bandwidth'].str.replace(' Mbps', '').str.replace(' Kbps', '').astype(float)
data.head()

,Timestamp,User_ID,Application_Type,Signal_Strength,Latency,Required_Bandwidth,Allocated_Bandwidth,Resource_Allocation
0,2023-09-03 10:00:00,User_1,Video_Call,-75.0,30.0,10.0,15.0,0.70
1,2023-09-03 10:00:00,User_2,Voice_Call,-80.0,20.0,100.0,120.0,0.80
2,2023-09-03 10:00:00,User_3,Streaming,-85.0,40.0,5.0,6.0,0.75
3,2023-09-03 10:00:00,User_4,Emergency_Service,-70.0,10.0,1.0,1.5,0.90
4,2023-09-03 10:00:00,User_5,Online_Gaming,-78.0,25.0,2.0,3.0,0.85


# Feature engineering

In [23]:
# Feature engineering
data['Bandwidth_Efficiency'] = data['Allocated_Bandwidth'] / data['Required_Bandwidth']
data['Signal_Quality_Category'] = pd.cut(data['Signal_Strength'], bins=[-100, -85, -70, 0], labels=['Weak', 'Moderate', 'Strong'])
data.head()

,Timestamp,User_ID,Application_Type,Signal_Strength,Latency,Required_Bandwidth,Allocated_Bandwidth,Resource_Allocation,Bandwidth_Efficiency,Signal_Quality_Category
0,2023-09-03 10:00:00,User_1,Video_Call,-75.0,30.0,10.0,15.0,0.70,1.5,Moderate
1,2023-09-03 10:00:00,User_2,Voice_Call,-80.0,20.0,100.0,120.0,0.80,1.2,Moderate
2,2023-09-03 10:00:00,User_3,Streaming,-85.0,40.0,5.0,6.0,0.75,1.2,Weak
3,2023-09-03 10:00:00,User_4,Emergency_Service,-70.0,10.0,1.0,1.5,0.90,1.5,Moderate
4,2023-09-03 10:00:00,User_5,Online_Gaming,-78.0,25.0,2.0,3.0,0.85,1.5,Moderate


In [24]:
data.head(20)

,Timestamp,User_ID,Application_Type,Signal_Strength,Latency,Required_Bandwidth,Allocated_Bandwidth,Resource_Allocation,Bandwidth_Efficiency,Signal_Quality_Category
0,2023-09-03 10:00:00,User_1,Video_Call,-75.0,30.0,10.0,15.0,0.70,1.500000,Moderate
1,2023-09-03 10:00:00,User_2,Voice_Call,-80.0,20.0,100.0,120.0,0.80,1.200000,Moderate
2,2023-09-03 10:00:00,User_3,Streaming,-85.0,40.0,5.0,6.0,0.75,1.200000,Weak
3,2023-09-03 10:00:00,User_4,Emergency_Service,-70.0,10.0,1.0,1.5,0.90,1.500000,Moderate
4,2023-09-03 10:00:00,User_5,Online_Gaming,-78.0,25.0,2.0,3.0,0.85,1.500000,Moderate
5,2023-09-03 10:00:00,User_6,Background_Download,-90.0,50.0,500.0,550.0,0.70,1.100000,Weak
6,2023-09-03 10:00:00,User_7,Web_Browsing,-88.0,30.0,1.0,1.0,0.60,1.000000,Weak
7,2023-09-03 10:00:00,User_8,IoT_Temperature,-95.0,100.0,10.0,15.0,0.50,1.500000,Weak
8,2023-09-03 10:00:00,User_9,Video_Streaming,-82.0,35.0,3.0,3.5,0.80,1.166667,Moderate
9,2023-09-03 10:00:00,User_10,File_Download,-75.0,45.0,2.0,2.0,0.70,1.000000,Moderate


In [25]:
# Define features and targets
features = ['Application_Type', 'Signal_Strength', 'Required_Bandwidth', 'Allocated_Bandwidth', 'Signal_Quality_Category']
target_latency = 'Latency'
target_efficiency = 'Bandwidth_Efficiency'

In [26]:
# Handling infinities and large values in the dataset
data = data.replace([np.inf, -np.inf], np.nan)
print(data.shape)
data = data.dropna(subset=['Latency', 'Bandwidth_Efficiency'])
print(data.shape)

(400, 10)
(393, 10)


In [27]:
# Define target variables
y_latency = data[target_latency]
y_efficiency = data[target_efficiency]

In [28]:
# Train-test split
X = data[features]
X_train_latency, X_test_latency, y_train_latency, y_test_latency = train_test_split(X, y_latency, test_size=0.2, random_state=42)
X_train_efficiency, X_test_efficiency, y_train_efficiency, y_test_efficiency = train_test_split(X, y_efficiency, test_size=0.2, random_state=42)

In [29]:
# Preprocessing pipelines
categorical_features = ['Application_Type', 'Signal_Quality_Category']
numerical_features = ['Signal_Strength', 'Required_Bandwidth', 'Allocated_Bandwidth']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])

In [30]:
# Model pipelines
models = {
    'Linear Regression': Pipeline(steps=[('preprocessor', preprocessor),
                                         ('regressor', LinearRegression())]),
    'Random Forest': Pipeline(steps=[('preprocessor', preprocessor),
                                      ('regressor', RandomForestRegressor(random_state=42))])
}

In [31]:
# set tracking server
mlflow.set_tracking_uri("http://3.107.14.172:5000/")
mlflow.set_experiment("QoS_Prediction")

<Experiment: artifact_location='mlflow-artifacts:/209821809856878572', creation_time=1733805188099, experiment_id='209821809856878572', last_update_time=1733805188099, lifecycle_stage='active', name='QoS_Prediction', tags={}>

In [32]:
# Model training and evaluation
results = {}
mlflow.sklearn.autolog()
for name, model in models.items():
  with mlflow.start_run(run_name=name+"Anshu"+str(np.random.randint(1000)),log_system_metrics=True) as run:
    print(f'Training {name} for Latency Prediction...')
    model.fit(X_train_latency, y_train_latency)
    y_pred_latency = model.predict(X_test_latency)

    print(f'Training {name} for Bandwidth Efficiency Prediction...')
    model.fit(X_train_efficiency, y_train_efficiency)
    y_pred_efficiency = model.predict(X_test_efficiency)

    # Store results
    results[name] = {
        'Latency': {
            'MAE': mean_absolute_error(y_test_latency, y_pred_latency),
            'RMSE': np.sqrt(mean_squared_error(y_test_latency, y_pred_latency)),
            'R2': r2_score(y_test_latency, y_pred_latency)
        },
        'Efficiency': {
            'MAE': mean_absolute_error(y_test_efficiency, y_pred_efficiency),
            'RMSE': np.sqrt(mean_squared_error(y_test_efficiency, y_pred_efficiency)),
            'R2': r2_score(y_test_efficiency, y_pred_efficiency)
        }
    }

# Display results
results

2024/12/11 01:21:20 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/12/11 01:21:20 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Training Linear Regression for Latency Prediction...
Training Linear Regression for Bandwidth Efficiency Prediction...
🏃 View run Linear RegressionAnshu399 at: http://3.107.14.172:5000/#/experiments/209821809856878572/runs/c4e5f4ce7bc84994a696db3290633773
🧪 View experiment at: http://3.107.14.172:5000/#/experiments/209821809856878572


2024/12/11 01:21:39 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/12/11 01:21:40 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
2024/12/11 01:21:40 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/12/11 01:21:40 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Training Random Forest for Latency Prediction...
Training Random Forest for Bandwidth Efficiency Prediction...
🏃 View run Random ForestAnshu454 at: http://3.107.14.172:5000/#/experiments/209821809856878572/runs/c9b06c795c644eadb5aba7ee3d90a4a1
🧪 View experiment at: http://3.107.14.172:5000/#/experiments/209821809856878572


2024/12/11 01:22:30 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/12/11 01:22:31 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


{'Linear Regression': {'Latency': {'MAE': 6.365094420185795,
   'RMSE': 7.946033854829205,
   'R2': 0.8076767901442934},
  'Efficiency': {'MAE': 0.1350199047317324,
   'RMSE': 0.21662304096277932,
   'R2': 0.00029450045440848616}},
 'Random Forest': {'Latency': {'MAE': 1.8966356238698008,
   'RMSE': 4.429782018774095,
   'R2': 0.9402283026952916},
  'Efficiency': {'MAE': 0.03321397254334228,
   'RMSE': 0.09458618553685297,
   'R2': 0.8094020574968452}}}

In [33]:
eval_data = X_test_latency
eval_data['target'] = y_test_latency

In [34]:
mlflow.set_experiment("Latency_Prediction")

<Experiment: artifact_location='mlflow-artifacts:/959328149099913042', creation_time=1733806956605, experiment_id='959328149099913042', last_update_time=1733806956605, lifecycle_stage='active', name='Latency_Prediction', tags={}>

In [35]:
# Model pipelines
models = {
    'Linear Regression': Pipeline(steps=[('preprocessor', preprocessor),
                                         ('regressor', LinearRegression())]),
    'Random Forest': Pipeline(steps=[('preprocessor', preprocessor),
                                      ('regressor', RandomForestRegressor(random_state=42,max_depth=12,min_samples_leaf=3))])
}

In [36]:
mlflow.sklearn.autolog()
for name, model in models.items():
  run_name = f"Latency {name} Anshu"+str(np.random.randint(1000))
  with mlflow.start_run(run_name=run_name,log_system_metrics=True) as run:
    print(f'Training {name} for Latency Prediction...')
    model.fit(X_train_latency, y_train_latency)

    mlflow.sklearn.log_model(model,run_name)
    model_uri = mlflow.get_artifact_uri(run_name)
    y_pred_latency = model.predict(X_test_latency)

    result = mlflow.evaluate(model_uri,eval_data,targets='target',model_type='regressor')

2024/12/11 01:22:41 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/12/11 01:22:41 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Training Linear Regression for Latency Prediction...


2024/12/11 01:22:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/12/11 01:22:58 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/12/11 01:22:58 WARNING mlflow.models.evaluation.evaluators.shap: Skip logging model explainability insights because the shap explainer None requires all feature values to be numeric, and each feature column must only contain scalar values.


🏃 View run Latency Linear Regression Anshu608 at: http://3.107.14.172:5000/#/experiments/959328149099913042/runs/4aaca202b4b1424294c3621df9d4f014
🧪 View experiment at: http://3.107.14.172:5000/#/experiments/959328149099913042


...

Training Random Forest for Latency Prediction...


2024/12/11 01:23:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/12/11 01:23:28 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/12/11 01:23:29 WARNING mlflow.models.evaluation.evaluators.shap: Skip logging model explainability insights because the shap explainer None requires all feature values to be numeric, and each feature column must only contain scalar values.


🏃 View run Latency Random Forest Anshu694 at: http://3.107.14.172:5000/#/experiments/959328149099913042/runs/ad8d58d977c34ea1b06de25464590ea1
🧪 View experiment at: http://3.107.14.172:5000/#/experiments/959328149099913042


2024/12/11 01:23:30 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/12/11 01:23:30 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


In [37]:
mlflow.set_experiment("Anshu-Latency-Pred")

2024/12/11 01:34:55 INFO mlflow.tracking.fluent: Experiment with name 'Anshu-Latency-Pred' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/794525821213039203', creation_time=1733880895627, experiment_id='794525821213039203', last_update_time=1733880895627, lifecycle_stage='active', name='Anshu-Latency-Pred', tags={}>

In [52]:
import time
eval_data = X_test_latency
eval_data['target'] = y_test_latency

from mlflow.models import infer_signature

mlflow.sklearn.autolog()
run_name = f"Latency "+str(np.random.randint(1000))
with mlflow.start_run(run_name=run_name,log_system_metrics=True) as run:
    start = time.time()
    model = Pipeline(steps=[('preprocessor', preprocessor),
                                      ('regressor', RandomForestRegressor(random_state=42,max_depth=12,min_samples_leaf=3))])
    print(f'Training {name} for Latency Prediction...')
    model.fit(X_train_latency, y_train_latency)

    signature = infer_signature(X_train_latency,y_train_latency)
    mlflow.sklearn.log_model(model,run_name,signature=signature)
    model_uri = mlflow.get_artifact_uri(run_name)
    result = mlflow.evaluate(model_uri,eval_data,targets='target',model_type='regressor',
                             evaluator_config={"metric_prefix": "test_",
                                               "log_model_explainability":True})
    end = time.time()
    mlflow.log_metric("time_of_execution",start-end)
    eval_data['predictions'] = model.predict(eval_data.drop(columns=["target"]))
    eval_data.to_csv("eval_data.csv")
    #mlflow.log_artifact("eval_data.csv",artifact_path="eval_data")

2024/12/11 02:15:37 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/12/11 02:15:37 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Training Random Forest for Latency Prediction...


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/12/11 02:16:04 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/12/11 02:16:04 WARNING mlflow.models.evaluation.evaluators.shap: Skip logging model explainability insights because the shap explainer None requires all feature values to be numeric, and each feature column must only contain scalar values.


🏃 View run Latency 483 at: http://3.107.14.172:5000/#/experiments/794525821213039203/runs/d1b42aee28c74404af23809659a1ddb5
🧪 View experiment at: http://3.107.14.172:5000/#/experiments/794525821213039203


2024/12/11 02:16:06 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/12/11 02:16:07 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


In [53]:
model_uri = mlflow.get_artifact_uri(run_name)
model_uri

'mlflow-artifacts:/794525821213039203/06bdacf4508a420c8d4c533877b8a6df/artifacts/Latency 483'

In [55]:
#mlflow.get_run(run_id=run.info.run_id)


In [66]:
from mlflow import MlflowClient
client = MlflowClient()
# create a registered model

registered_model_name = "Anshu-Latency-Pred2"
client.create_registered_model(registered_model_name)

client.create_model_version(name=registered_model_name,
                            source = f"runs:/{run.info.run_id}/model",
                            run_id=run.info.run_id)

2024/12/11 02:45:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Anshu-Latency-Pred2, version 1


<ModelVersion: aliases=[], creation_timestamp=1733885104619, current_stage='None', description='', last_updated_timestamp=1733885104619, name='Anshu-Latency-Pred2', run_id='d1b42aee28c74404af23809659a1ddb5', run_link='', source='runs:/d1b42aee28c74404af23809659a1ddb5/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [67]:
client.set_registered_model_alias(registered_model_name,"production","1")